# Predictive models for NFL games to enable spread betting.

## Load packages and data files

In [94]:
# basic python packages
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, accuracy_score,  r2_score
from sklearn.preprocessing import StandardScaler

In [2]:
# regression model packages
import xgboost as xgb
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [62]:
#import data
df_results = pd.read_csv('../data/nfl_games_pfr.csv')

## Create rankings using pagerank from nx

In [112]:
# Graph Functions
def create_graph(game_data, week_num):
    G = nx.DiGraph()
    margin_totals = defaultdict(float)
    game_counts = defaultdict(int)
    
    # Iterate over each game result
    for _, row in game_data.iterrows():
        winner = row['winner']
        loser = row['loser']
        week = row['week']
        margin = row['margin']

        key = (loser, winner)
        margin_totals[key] += margin
        game_counts[key] += 1

    # Add edges with average margin as weight
    for (loser, winner), total_margin in margin_totals.items():
        avg_margin = total_margin / game_counts[(loser, winner)]
        G.add_edge(loser, winner, weight=avg_margin)
    
    return G

# Function to calculate rankings based on the weighted graph
def calculate_rankings(graph):
    # Calculate PageRank with weights
    return pd.DataFrame(nx.pagerank(graph, alpha=0.92, weight='weight').items(), columns=['Team', 'Ranking'])

# Function to generate rankings for each season and week
def generate_rankings(df_filtered, feature_type):
    # Initialize an empty list to store rankings DataFrames
    ranking_dfs = []

    # Loop over each distinct season in the dataset
    for season in df_filtered['season'].unique():
        # Filter the game data for the current season
        season_data = df_filtered[df_filtered['season'] == season]
        
        # Loop over the weeks for this season
        for week in range(2, season_data['week'].max() + 1):
            # Filter the game data up to the current week for the current season
            filtered_data = season_data[season_data['week'] <= week]
            
            # Create the directed graph for the current season and weeks
            G = create_graph(filtered_data, week)
            
            # Calculate the rankings based on the weighted graph
            rankings = calculate_rankings(G)

            # Round the rankings to 4 decimal places
            rankings['Ranking'] = rankings['Ranking'].round(5)
            
            # Add columns indicating the season and week, with a leading zero for weeks
            rankings['SeasonWeek'] = f"{season}_W{str(week).zfill(2)}"
            rankings['Season'] = season
            rankings['Week'] = week
            rankings['Type'] = feature_type
            
            # Append the rankings to the list
            ranking_dfs.append(rankings)
    
    # Concatenate all rankings DataFrames into a single DataFrame
    return pd.concat(ranking_dfs)

print("DONE")

DONE


In [113]:
### OFFENSE ###
# Filtered data frames for each feature set
df_offense = df_results[~df_results['Margin Yds'].isna()][['season','week','Winner Yds','Loser Yds','Margin Yds']]
df_offense = df_offense.rename(columns={'Winner Yds':'winner', 'Loser Yds':'loser','Margin Yds':'margin'})

# Generate rankings for each feature type
final_offense_rankings_df = generate_rankings(df_offense, "Offense")
print(final_offense_rankings_df.head())
print(len(final_offense_rankings_df))

  Team  Ranking SeasonWeek  Season  Week     Type
0   KC  0.01344   2024_W02    2024     2  Offense
1  BAL  0.03328   2024_W02    2024     2  Offense
2  PHI  0.01344   2024_W02    2024     2  Offense
3   GB  0.01900   2024_W02    2024     2  Offense
4  ATL  0.02375   2024_W02    2024     2  Offense
4832


In [114]:
### DEFENSE ###
# Filtered data frames for each feature set
df_defense = df_results[~df_results['Margin Yds'].isna()][['season','week','Winner TO','Loser TO','Margin TO']]
df_defense = df_defense.rename(columns={'Winner TO':'winner', 'Loser TO':'loser','Margin TO':'margin'})

# Generate rankings for each feature type
final_defense_rankings_df = generate_rankings(df_defense, "Defense")
print(final_defense_rankings_df.head())
print(len(final_defense_rankings_df))

  Team  Ranking SeasonWeek  Season  Week     Type
0  BAL  0.01844   2024_W02    2024     2  Defense
1   KC  0.05100   2024_W02    2024     2  Defense
2   GB  0.01844   2024_W02    2024     2  Defense
3  PHI  0.05324   2024_W02    2024     2  Defense
4  PIT  0.01844   2024_W02    2024     2  Defense
4832


In [115]:
### SCORE ###
# Filtered data frames for each feature set
df_score = df_results[~df_results['Margin Yds'].isna()][['season','week','Winner Abbr','Loser Abbr','Margin Pts']]
df_score = df_score.rename(columns={'Winner Abbr':'winner', 'Loser Abbr':'loser', 'Margin Pts':'margin'})

# Generate rankings for each feature type
final_score_rankings_df = generate_rankings(df_score, "Score")
print(final_score_rankings_df.head())
print(len(final_score_rankings_df))

  Team  Ranking SeasonWeek  Season  Week   Type
0  BAL  0.01573   2024_W02    2024     2  Score
1   KC  0.02793   2024_W02    2024     2  Score
2   GB  0.02658   2024_W02    2024     2  Score
3  PHI  0.04018   2024_W02    2024     2  Score
4  ATL  0.05270   2024_W02    2024     2  Score
4832


In [116]:
# Concatenate all feature rankings into a single DataFrame
final_rankings_df = pd.concat([final_offense_rankings_df, final_score_rankings_df, final_defense_rankings_df])
print(final_rankings_df.head())
print(len(final_rankings_df))
final_rankings_df.to_csv('../data/nfl_rankings_combined.csv')

  Team  Ranking SeasonWeek  Season  Week     Type
0   KC  0.01344   2024_W02    2024     2  Offense
1  BAL  0.03328   2024_W02    2024     2  Offense
2  PHI  0.01344   2024_W02    2024     2  Offense
3   GB  0.01900   2024_W02    2024     2  Offense
4  ATL  0.02375   2024_W02    2024     2  Offense
14496


## Create additional features and interaction variables

In [117]:
# Preprocess the combined_rankings dataframe to create offensive and defensive rank features
rankings = final_rankings_df.pivot(index=['Team', 'SeasonWeek', 'Season', 'Week'], columns='Type', values='Ranking').reset_index()
rankings.rename(columns={'Offense': 'OffenseRank', 'Defense': 'DefenseRank', 'Score': 'ScoreRank'}, inplace=True)

# Adjust rankings to use the prior week's data
rankings['Week'] += 1

# Merge the rankings with game results to create the feature set
def merge_rankings(df, team_column, prefix):
    return df.merge(rankings, left_on=[team_column, 'season', 'week'], right_on=['Team', 'Season', 'Week'], how='left') \
             .rename(columns={'OffenseRank': f'{prefix}_OffenseRank',
                              'DefenseRank': f'{prefix}_DefenseRank',
                              'ScoreRank': f'{prefix}_ScoreRank'})

merged_df = merge_rankings(df_results, 'Home Team', 'Home')
merged_df = merge_rankings(merged_df, 'Away Team', 'Away')

# Drop unnecessary columns
merged_df.drop(columns=['Unnamed: 0', 'Team_x', 'Season_x', 'SeasonWeek_x', 'Week_x',
                        'Team_y', 'Season_y', 'SeasonWeek_y', 'Week_y'], inplace=True)

# Filter to only games where rankings exist for both teams
merged_df.dropna(subset=['Home_OffenseRank', 'Home_DefenseRank', 'Away_OffenseRank', 'Away_DefenseRank'], inplace=True)

# Create target variable (spread)
merged_df['Spread'] = merged_df.apply(lambda row: row['PtsW'] - row['PtsL']
                                      if row['Home Team'] == row['Winner Abbr']
                                      else row['PtsL'] - row['PtsW'], axis=1)

# Create feature deltas and ratios
rank_features = ['OffenseRank', 'DefenseRank', 'ScoreRank']
for feature in rank_features:
    merged_df[f'Delta_{feature}'] = merged_df[f'Home_{feature}'] - merged_df[f'Away_{feature}']
    merged_df[f'Ratio_{feature}'] = merged_df[f'Home_{feature}'] / (merged_df[f'Away_{feature}'] + 1e-5)

# Create interaction and quadratic terms
interaction_terms = [
    ('Home_OffenseRank', 'Away_DefenseRank'),
    ('Away_OffenseRank', 'Home_DefenseRank'),
    ('Home_OffenseRank', 'Away_OffenseRank'),
    ('Away_DefenseRank', 'Home_DefenseRank')
]
for i, (col1, col2) in enumerate(interaction_terms, 1):
    merged_df[f'intTerm{i}'] = merged_df[col1] * merged_df[col2]

quadratic_terms = [
    'Home_OffenseRank', 'Away_OffenseRank', 'Home_DefenseRank', 'Away_DefenseRank'
]
for i, col in enumerate(quadratic_terms, 1):
    merged_df[f'quadTerm{i}'] = merged_df[col] ** 2

# Create strength features
merged_df['HomeStrength'] = merged_df['Home_OffenseRank'] + merged_df['Home_DefenseRank'] + merged_df['Home_ScoreRank']
merged_df['AwayStrength'] = merged_df['Away_OffenseRank'] + merged_df['Away_DefenseRank'] + merged_df['Away_ScoreRank']

# Create variable for historical and future games
merged_df['GameType'] = merged_df.apply(lambda row: "Historical" 
                                      if not pd.isna(row['Margin Yds'])
                                      else "Future", axis=1)

print(merged_df.columns)
merged_df.to_csv('../data/nfl_games_pfr_features.csv')

#Separate played and upcoming games
merged_played = merged_df[~merged_df['Margin Yds'].isna()]
merged_upcoming = merged_df[merged_df['Margin Yds'].isna()]
print(len(merged_played),len(merged_upcoming))

Index(['Day', 'Date', 'Time', 'Winner', 'LoserIsHome', 'Loser', 'PtsW', 'PtsL',
       'YdsW', 'TOW', 'YdsL', 'TOL', 'season', 'week', 'Winner Abbr',
       'Loser Abbr', 'Home Team', 'Away Team', 'Winner Yds', 'Loser Yds',
       'Margin Yds', 'Winner TO', 'Loser TO', 'Margin TO', 'Margin Pts',
       'Total Pts', 'Margin Pct', 'home_team_winner', 'away_win_bonus',
       'Home_DefenseRank', 'Home_OffenseRank', 'Home_ScoreRank', 'Week',
       'Away_DefenseRank', 'Away_OffenseRank', 'Away_ScoreRank', 'Spread',
       'Delta_OffenseRank', 'Ratio_OffenseRank', 'Delta_DefenseRank',
       'Ratio_DefenseRank', 'Delta_ScoreRank', 'Ratio_ScoreRank', 'intTerm1',
       'intTerm2', 'intTerm3', 'intTerm4', 'quadTerm1', 'quadTerm2',
       'quadTerm3', 'quadTerm4', 'HomeStrength', 'AwayStrength', 'GameType'],
      dtype='object')
2097 13


## Model and basic feature evaluation

In [95]:
# Function to train the model
def train_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

scaler = StandardScaler()

In [123]:
# Define features and target
feature_sets = [
    merged_played[['Home_ScoreRank','Away_ScoreRank']], 
    merged_played[['Delta_ScoreRank']],
    merged_played[['Home_OffenseRank', 'Home_DefenseRank','Home_ScoreRank', 'Away_OffenseRank', 'Away_DefenseRank','Away_ScoreRank']],
    merged_played[['Delta_ScoreRank','Delta_OffenseRank','Delta_DefenseRank']],
    merged_played[['Home_OffenseRank', 'Home_DefenseRank','Away_OffenseRank', 'Away_DefenseRank']],
    merged_played[['Delta_OffenseRank','Delta_DefenseRank']],
    merged_played[['Home_OffenseRank', 'Home_DefenseRank','Home_ScoreRank', 'Away_OffenseRank',\
               'Away_DefenseRank','Away_ScoreRank', 'Delta_ScoreRank','Delta_OffenseRank',\
                'Delta_DefenseRank', 'Ratio_ScoreRank', 'Ratio_OffenseRank', 'Ratio_DefenseRank',\
                'intTerm1', 'intTerm2','intTerm3', 'intTerm4', \
                'quadTerm1','quadTerm2', 'quadTerm3', 'quadTerm4', \
                'HomeStrength', 'AwayStrength']],
    merged_played[['Delta_ScoreRank','Delta_OffenseRank','Delta_DefenseRank',\
                'intTerm1', 'intTerm2','intTerm3', 'intTerm4', \
                'quadTerm1','quadTerm2', 'quadTerm3', 'quadTerm4']],
    merged_played[['Delta_ScoreRank','Delta_OffenseRank','Delta_DefenseRank',\
                'intTerm1', 'intTerm2','intTerm3', 'intTerm4']],
    merged_played[['Delta_ScoreRank',\
                'intTerm1', 'intTerm2','intTerm3', 'intTerm4']],
    scaler.fit_transform(merged_played[['Delta_ScoreRank','Delta_OffenseRank','Delta_DefenseRank', \
                'intTerm1', 'intTerm2','intTerm3', 'intTerm4']]),
    scaler.fit_transform(merged_played[['Delta_ScoreRank','Delta_OffenseRank','Delta_DefenseRank', \
                'intTerm1', 'intTerm2','intTerm3', 'intTerm4', \
                'quadTerm1','quadTerm2', 'quadTerm3', 'quadTerm4']])

]
y = merged_played['Spread']

# Define models
models = [
    ('LM', LinearRegression()),
    ('LMnoINT', LinearRegression(fit_intercept=False)),
    ('Ridge0.05', Ridge(alpha=0.05)),
    ('Ridge0.1', Ridge(alpha=0.1)),
    ('Ridge0.5', Ridge(alpha=0.5)),  # You can tune the alpha parameter
    ('XGBoostBase', xgb.XGBRegressor(objective='reg:squarederror', n_estimators=50, learning_rate=0.1, max_depth=3, random_state=42)),
    ('XGBoost150', xgb.XGBRegressor(objective='reg:squarederror', n_estimators=150, learning_rate=0.1, max_depth=3, random_state=42)),
    ('XGBoost5', xgb.XGBRegressor(objective='reg:squarederror', n_estimators=50, learning_rate=0.1, max_depth=5, random_state=42)),
    ('XGBoost0.5', xgb.XGBRegressor(objective='reg:squarederror', n_estimators=50, learning_rate=0.05, max_depth=3, random_state=42)),
    ('RF50', RandomForestRegressor(n_estimators=50, max_depth=3, random_state=42)),
    ('RF10', RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42)),
    ('SVR', SVR(kernel='rbf', C=1.0, epsilon=0.1)),
    ('SVRlin', SVR(kernel='linear', C=1.0, epsilon=0.1)),
    ('kNN5', KNeighborsRegressor(n_neighbors=5)),
    ('kNN17', KNeighborsRegressor(n_neighbors=17))
]

print("DONE")

DONE


In [124]:
# DataFrame to store results
results = pd.DataFrame(columns=['Model', 'Feature_Set', 'MSE', 'Accuracy'])

# Iterate over feature sets and models
for i, X in enumerate(feature_sets):
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for model_name, model in models:
        print(f"Evaluating model: {model_name} with feature set {i+1}")
        model = train_model(model, X_train, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test)

        # Make predictions on the training set
        y_train_pred = model.predict(X_train)

        # Evaluate the model
        mse = mean_squared_error(y_test, y_pred)
        #print(f"Mean Squared Error: {mse}")

        '''  MUST HAVE DELTA_SCORE RANK
        # Calculate accuracy of predicting the winner for Delta_ScoreRank > 0.02
        subset_idx = X_test['Delta_ScoreRank'].abs() > 0.02
        subset_y_test = y_test[subset_idx]
        subset_y_pred = y_pred[subset_idx]
        y_pred_winner = ['Home' if pred > 0 else 'Away' for pred in subset_y_pred]
        y_test_winner = ['Home' if actual > 0 else 'Away' for actual in subset_y_test]
        accuracy = accuracy_score(y_test_winner, y_pred_winner)
        print(f"Winner Prediction Accuracy (|Delta_ScoreRank| > 0.02): {accuracy * 100:.2f}%\n")
        '''

        # Calculate accuracy of predicting the winner
        y_pred_winner = ['Home' if pred > 0 else 'Away' for pred in y_pred]
        y_test_winner = ['Home' if actual > 0 else 'Away' for actual in y_test]
        accuracy = accuracy_score(y_test_winner, y_pred_winner)
        #print(f"Winner Prediction Accuracy: {accuracy * 100:.2f}%\n")

        # Store the results
        new_result = pd.DataFrame({
            'Model': [model_name],
            'Feature_Set': [f'Set {i+1}'],
            'MSE': [mse],
            'Accuracy': [accuracy * 100]
        })
        results = pd.concat([results, new_result], ignore_index=True)

# Print the results DataFrame
#print(results)

Evaluating model: LM with feature set 1
Evaluating model: LMnoINT with feature set 1
Evaluating model: Ridge0.05 with feature set 1
Evaluating model: Ridge0.1 with feature set 1
Evaluating model: Ridge0.5 with feature set 1
Evaluating model: XGBoostBase with feature set 1
Evaluating model: XGBoost150 with feature set 1
Evaluating model: XGBoost5 with feature set 1


C:\Users\see_w\AppData\Local\Temp\ipykernel_17132\568564045.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_result], ignore_index=True)


Evaluating model: XGBoost0.5 with feature set 1
Evaluating model: RF50 with feature set 1
Evaluating model: RF10 with feature set 1
Evaluating model: SVR with feature set 1
Evaluating model: SVRlin with feature set 1
Evaluating model: kNN5 with feature set 1
Evaluating model: kNN17 with feature set 1
Evaluating model: LM with feature set 2
Evaluating model: LMnoINT with feature set 2
Evaluating model: Ridge0.05 with feature set 2
Evaluating model: Ridge0.1 with feature set 2
Evaluating model: Ridge0.5 with feature set 2
Evaluating model: XGBoostBase with feature set 2
Evaluating model: XGBoost150 with feature set 2
Evaluating model: XGBoost5 with feature set 2
Evaluating model: XGBoost0.5 with feature set 2
Evaluating model: RF50 with feature set 2
Evaluating model: RF10 with feature set 2
Evaluating model: SVR with feature set 2
Evaluating model: SVRlin with feature set 2
Evaluating model: kNN5 with feature set 2
Evaluating model: kNN17 with feature set 2
Evaluating model: LM with fea

In [ ]:
# Print the results DataFrame
print(results.to_csv(index=False))

## Optimize the best model and feature set

Linear Regression
Factors:  ['Delta_ScoreRank','Delta_OffenseRank','Delta_DefenseRank', 'intTerm1', 'intTerm2','intTerm3', 'intTerm4',
                'quadTerm1','quadTerm2', 'quadTerm3', 'quadTerm4']

In [144]:
filtered_played = merged_played[merged_played['Delta_ScoreRank'].abs() > 0.02]
X = filtered_played[['Delta_ScoreRank',\
                'intTerm1', 'intTerm2','intTerm3', 'intTerm4', \
                'quadTerm1','quadTerm2', 'quadTerm3', 'quadTerm4']]
y = filtered_played['Spread']

X = sm.add_constant(X)
    
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the linear regression model using statsmodels to see p-values
model = sm.OLS(y_train, X_train).fit()
print(f"\nModel {i} Summary:\n")
print(model.summary())


Model 11 Summary:

                            OLS Regression Results                            
Dep. Variable:                 Spread   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     11.65
Date:                Mon, 07 Oct 2024   Prob (F-statistic):           2.85e-17
Time:                        11:58:23   Log-Likelihood:                -3153.2
No. Observations:                 780   AIC:                             6326.
Df Residuals:                     770   BIC:                             6373.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.

In [145]:
from sklearn.feature_selection import RFE

model = LinearRegression()
rfe = RFE(model, n_features_to_select=5)
rfe = rfe.fit(X, y)
print(rfe.support_)
print(rfe.ranking_)

from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=5)
X_new = selector.fit_transform(X, y)
print(selector.get_support())

[False False  True  True  True  True False False False  True]
[6 3 1 1 1 1 2 4 5 1]
[False  True  True  True False False False  True  True False]


In [137]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = merged_played[['Delta_ScoreRank',
                   'intTerm1', 'intTerm2', 'intTerm3', 'intTerm4',
                   'quadTerm1', 'quadTerm2', 'quadTerm3', 'quadTerm4']]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif)

   VIF Factor         features
0    1.145290  Delta_ScoreRank
1    4.347253         intTerm1
2    3.591348         intTerm2
3    3.143727         intTerm3
4    4.325440         intTerm4
5    2.169210        quadTerm1
6    2.287461        quadTerm2
7    3.118295        quadTerm3
8    2.838891        quadTerm4
